In [84]:
import cv2
from ultralytics import YOLO
import cvzone
import math

In [85]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [86]:
model = YOLO('./Yolo-Weights/yolov8l.pt')

In [87]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [88]:
while True:
    success, img = cap.read()
    results = model(img,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
    cv2.imshow("Image",img)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 739.7ms
Speed: 3.5ms preprocess, 739.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



0.88
0.85
0.85
0.83
0.7
0.68
0.66
0.64
0.57
0.53
0.5
0.44
0.38
0.37
0.36
0.35
0.28
0.27


0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 779.7ms
Speed: 3.1ms preprocess, 779.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



0.9
0.85
0.84
0.84
0.83
0.69
0.65
0.62
0.56
0.53
0.42
0.42
0.41
0.39
0.35
0.31
0.3
0.29


0: 384x640 1 person, 16 cars, 1 truck, 1 traffic light, 1 stop sign, 709.8ms
Speed: 2.3ms preprocess, 709.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



...

0.89
0.84
0.82
0.78
0.77
0.77
0.66
0.5
0.45
0.45
0.37
0.32
0.29
0.28
0.27
0.27
0.26


0: 384x640 1 person, 11 cars, 1 motorcycle, 2 trucks, 715.1ms
Speed: 3.9ms preprocess, 715.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



0.87
0.85
0.84
0.79
0.79
0.66
0.58
0.57
0.55
0.54
0.5
0.43
0.43
0.39
0.35


0: 384x640 1 person, 12 cars, 1 truck, 682.5ms
Speed: 2.0ms preprocess, 682.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


0.88
0.84
0.83
0.81
0.8
0.77
0.69
0.52
0.42
0.39
0.35
0.34
0.34
0.34
